## CLIENT INFERENCE ON-LINE

## Libray Import

In [1]:
#!pip install imblearn

### LLAMAR AL MODELO

In [2]:
import ray
from ray import serve

import mlflow
import pandas as pd

from fastapi import FastAPI, Query
from pydantic import BaseModel#, Field
from fastapi.responses import HTMLResponse
from fastapi.responses import RedirectResponse

import optbinning
import imblearn
#import funpymodeling
import psycopg2
from sqlalchemy import create_engine

# librerias creadas
# from mlflow_ import mlflow_

In [3]:
import pandas as pd
import requests

## Read DAMs to Predict

In [4]:
#DATASET_SELECTED = pd.read_csv('../data/raw/MLOps_Set_pruebas_Modelo_Nacional_Exportacion.csv')
DATASET_SELECTED = pd.read_parquet('../data/processed/MLOps_Set_pruebas_Modelo_Nacional_Exportacion.parquet')
DATASET_SELECTED = DATASET_SELECTED.loc[:, DATASET_SELECTED.columns != 'target']
DATASET_SELECTED=DATASET_SELECTED.sample(4)
DATASET_SELECTED.head(4)

,capitulo_mxf_t0052,cnt_pes_brut_t0051,cnt_totserie_t0051,cod_puertemb_t0051,cod_viatrans_t0051,c390,c412,c463,ddp_ciiu,dia_ant_dam_altruc,...,mto_fob_mfs,mto_tovaladu_t0051,mto_itf_tot_partic,mto_itf_esf_partic,part_prov_high,part_prov_medium,part_prov_outlier,rat_fob_pneto_mfs,ruc_agente_dam,seccion
3,03,843.0,1.0,PECLL,4,2.912794e+08,1172026.0,0.0,05002,3246.0,...,6464.88,6464.88,10214.88,20399.10,0.038410,0.176961,0.675536,8.517628,20545334462,01
11,61,717.0,2.0,PELIM,4,1.472413e+08,10148170.0,101310475.0,18100,10716.0,...,29628.30,52075.20,4013.53,7097.04,0.036998,0.047597,0.836166,86.345899,20510298013,11
1,26,10500020.0,1.0,PEMRI,1,2.904253e+10,93335772.0,185811593.0,13200,10510.0,...,15097859.44,15097859.44,9438.40,9445.47,0.020342,0.020378,0.953201,1.437889,20127705969,05
19,83,340.0,1.0,PEILQ,3,2.172630e+06,69171.0,13201743.0,51313,831.0,...,49686.00,49686.00,23.80,49.95,0.112124,0.126839,0.724735,147.000000,20532620580,15


In [5]:
request = DATASET_SELECTED.head(4).to_dict('records')
request

[{'capitulo_mxf_t0052': '03',
  'cnt_pes_brut_t0051': 843.0,
  'cnt_totserie_t0051': 1.0,
  'cod_puertemb_t0051': 'PECLL',
  'cod_viatrans_t0051': '4',
  'c390': 291279409.0,
  'c412': 1172026.0,
  'c463': 0.0,
  'ddp_ciiu': '05002',
  'dia_ant_dam_altruc': 3246.0,
  'margen_neto': 0.0,
  'mto_fob_mfs': 6464.88,
  'mto_tovaladu_t0051': 6464.88,
  'mto_itf_tot_partic': 10214.88,
  'mto_itf_esf_partic': 20399.1,
  'part_prov_high': 0.03841,
  'part_prov_medium': 0.176961,
  'part_prov_outlier': 0.675536,
  'rat_fob_pneto_mfs': 8.517628,
  'ruc_agente_dam': '20545334462',
  'seccion': '01'},
 {'capitulo_mxf_t0052': '61',
  'cnt_pes_brut_t0051': 717.0,
  'cnt_totserie_t0051': 2.0,
  'cod_puertemb_t0051': 'PELIM',
  'cod_viatrans_t0051': '4',
  'c390': 147241343.0,
  'c412': 10148170.0,
  'c463': 101310475.0,
  'ddp_ciiu': '18100',
  'dia_ant_dam_altruc': 10716.0,
  'margen_neto': 0.00635,
  'mto_fob_mfs': 29628.3,
  'mto_tovaladu_t0051': 52075.2,
  'mto_itf_tot_partic': 4013.53,
  'mto_itf

## Request

In [7]:
for i in range(len(DATASET_SELECTED)):
    response= requests.post("http://172.26.56.65:8000/v1/gestionriesgo/evaluacionriesgoaduanero/sobrevalu-expo-nac/score/e/damscore", json=request[i]) #LOCAL
    result = response.json()
    print(request[i], "--> ", result)
    print("-"*20)

{'capitulo_mxf_t0052': '03', 'cnt_pes_brut_t0051': 843.0, 'cnt_totserie_t0051': 1.0, 'cod_puertemb_t0051': 'PECLL', 'cod_viatrans_t0051': '4', 'c390': 291279409.0, 'c412': 1172026.0, 'c463': 0.0, 'ddp_ciiu': '05002', 'dia_ant_dam_altruc': 3246.0, 'margen_neto': 0.0, 'mto_fob_mfs': 6464.88, 'mto_tovaladu_t0051': 6464.88, 'mto_itf_tot_partic': 10214.88, 'mto_itf_esf_partic': 20399.1, 'part_prov_high': 0.03841, 'part_prov_medium': 0.176961, 'part_prov_outlier': 0.675536, 'rat_fob_pneto_mfs': 8.517628, 'ruc_agente_dam': '20545334462', 'seccion': '01'} -->  {'cod': 200, 'msg': 'Score calculated successfully.', 'data': {'em_eventprobability': 0.6238251134910107}, 'errors': None}
--------------------
{'capitulo_mxf_t0052': '61', 'cnt_pes_brut_t0051': 717.0, 'cnt_totserie_t0051': 2.0, 'cod_puertemb_t0051': 'PELIM', 'cod_viatrans_t0051': '4', 'c390': 147241343.0, 'c412': 10148170.0, 'c463': 101310475.0, 'ddp_ciiu': '18100', 'dia_ant_dam_altruc': 10716.0, 'margen_neto': 0.00635, 'mto_fob_mfs': 2